# Image Recogniser Model

# 1. Overview:

#### 1.1 Team:
- Emma
- Raphael

#### 1.2 Topic:
**Garbage Image Classification**
##### 1.2.1 Objective:
- Build model to classify different types of garbage (recycable, non-recycable) 
    - MVP: linear classifier, SVM, DF or pretrained CNN
    - next level: self trained CNN
- Dive into CNN
- optional: create APP

##### 1.2.2 Data:
Collected by Gary Thung and Mindy Yang.
The Dataset contains 2527 images of six classes of garbage deployed on plain background:
 - 501 glass
 - 594 paper
 - 403 cardboard
 - 482 plastic
 - 410 metal
 - 137 trash


##### 1.2.3 Link to the Dataset:
https://github.com/garythung/trashnet/blob/master/data/dataset-resized.zip


## 2. Import libraries <a name="2"></a>
Import necessary libraries for data-handling, plotting, modeling.
[↑ back to top ↑](#index)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
#import pytorch_lightning

import pickle
from tqdm import trange, tqdm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 3. Load data <a name="3"></a>
[↑ back to top ↑](#index)

#### 3.1 Load and transform data:

In [7]:
data_transformer = transforms.Compose([#transforms.RandomResizedCrop(224), 
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
#                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                        ])

data_set = datasets.ImageFolder(root='data/dataset-resized', transform=data_transformer)

loader = torch.utils.data.DataLoader(data_set, batch_size=36, shuffle=True)

test_set, valid_set = torch.utils.data.random_split(data_set, (1769, 758))

labels = data_set.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [8]:
device

device(type='cpu')

#### 3.2 Visualise transformed images

In [9]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(16,16))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
images, classes = next(iter(loader))

# Make a grid from batch
#batch_grid = torchvision.utils.make_grid(images)

#imshow(batch_grid, title=[labels[x] for x in classes])


In [10]:
# grid = torchvision.utils.make_grid(images, nrow=8)

# plt.figure(figsize=(15, 15))
# plt.imshow(np.transpose(grid, (1, 2, 0)))

# print('labels: ', [labels[x] for x in classes])

## 4. Create model <a name="4"></a>
[↑ back to top ↑](#index)

In [11]:
try:
    with open("model/resnet18.pickle","rb") as fp:
        model = pickle.load(fp)
except:
    print("Couldn't find model")
    model = models.resnet18(pretrained=True)

## 5. Train model <a name="5"></a>
Improve the training of the model using our own data set.
[↑ back to top ↑](#index)

In [18]:
# tuning parameters
learn_rate = 1e-3
epochs = 3

In [19]:
# Initialise the loss function (difference between likelihood of predicted label and actual label)

loss = nn.CrossEntropyLoss() # combination of mse and negative log likelihood

In [20]:
optimiser = torch.optim.SGD(model.parameters(), lr=learn_rate)

In [21]:
def training_loop(loader, model, loss, optimiser):
    
    total_loss = 0
    
    for batch, (images, labels) in enumerate(loader):
        images = images.to(device)
        labels = labels.to(device)
        pred = model(images)
        loss = nn.functional.cross_entropy(pred, labels)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        print(f"Batch {batch + 1} loss: {loss}") 
        
        total_loss += loss.item()
    
    print(f"Accumulated loss: {total_loss}") 

    return 
    

In [22]:
for i in trange(epochs):
    
    print()
    print(f"Epoch {i+1}\n------------------------")
    training_loop(loader, model, loss, optimiser)
    with open('model/resnet18.pickle','wb') as fp:
        pickle.dump(model, fp)
    

  0%|          | 0/30 [00:00<?, ?it/s]


Epoch 1
------------------------
Batch 1 loss: 0.6867982745170593
Batch 2 loss: 0.5414140224456787
Batch 3 loss: 0.4359928071498871
Batch 4 loss: 0.8182901740074158
Batch 5 loss: 0.6527044773101807
Batch 6 loss: 0.3561745882034302
Batch 7 loss: 0.45623859763145447
Batch 8 loss: 0.5232836604118347
Batch 9 loss: 1.0272841453552246
Batch 10 loss: 0.6115518808364868
Batch 11 loss: 0.6043291091918945
Batch 12 loss: 0.7603785395622253
Batch 13 loss: 0.6321735382080078
Batch 14 loss: 0.5346982479095459
Batch 15 loss: 0.46713292598724365
Batch 16 loss: 0.5995666980743408
Batch 17 loss: 0.5790975093841553
Batch 18 loss: 0.6190906167030334
Batch 19 loss: 0.5384702086448669
Batch 20 loss: 0.578984797000885
Batch 21 loss: 0.7049403786659241
Batch 22 loss: 0.32998135685920715
Batch 23 loss: 0.5392200946807861
Batch 24 loss: 0.6354419589042664
Batch 25 loss: 0.4477543234825134
Batch 26 loss: 0.6336253881454468
Batch 27 loss: 0.4891048073768616
Batch 28 loss: 0.5785900354385376
Batch 29 loss: 0.6765

  3%|▎         | 1/30 [19:17<9:19:32, 1157.67s/it]

Batch 71 loss: 1.270774245262146
Accumulated loss: 42.100328236818314

Epoch 2
------------------------
Batch 1 loss: 0.46658769249916077
Batch 2 loss: 0.6015125513076782
Batch 3 loss: 0.49798691272735596
Batch 4 loss: 0.5877109169960022
Batch 5 loss: 0.612987756729126
Batch 6 loss: 0.5670445561408997
Batch 7 loss: 0.719629168510437
Batch 8 loss: 0.40763962268829346
Batch 9 loss: 0.44139301776885986
Batch 10 loss: 0.40496835112571716
Batch 11 loss: 0.5562834143638611
Batch 12 loss: 0.5620580315589905
Batch 13 loss: 0.3609175682067871
Batch 14 loss: 0.6048715114593506
Batch 15 loss: 0.5190971493721008
Batch 16 loss: 0.553987979888916
Batch 17 loss: 0.6544433832168579
Batch 18 loss: 0.6230928897857666
Batch 19 loss: 0.6637247204780579
Batch 20 loss: 0.5681900978088379
Batch 21 loss: 0.41524195671081543
Batch 22 loss: 0.6177259087562561
Batch 23 loss: 0.5049843788146973
Batch 24 loss: 0.5302884578704834
Batch 25 loss: 0.40644320845603943
Batch 26 loss: 0.5283986926078796
Batch 27 loss: 0.

  7%|▋         | 2/30 [37:25<8:41:11, 1116.82s/it]

...

Batch 71 loss: 0.8716816306114197
Accumulated loss: 32.81863008439541

Epoch 4
------------------------
Batch 1 loss: 0.511759877204895
Batch 2 loss: 0.3905051648616791
Batch 3 loss: 0.44187837839126587
Batch 4 loss: 0.5280813574790955
Batch 5 loss: 0.5575722455978394
Batch 6 loss: 0.39553844928741455
Batch 7 loss: 0.5635295510292053
Batch 8 loss: 0.5429761409759521
Batch 9 loss: 0.4953882396221161
Batch 10 loss: 0.2869512736797333
Batch 11 loss: 0.617440402507782
Batch 12 loss: 0.684714674949646
Batch 13 loss: 0.4206211566925049
Batch 14 loss: 0.3333183825016022
Batch 15 loss: 0.6790034174919128
Batch 16 loss: 0.31417274475097656
Batch 17 loss: 0.435843825340271
Batch 18 loss: 0.4426368772983551
Batch 19 loss: 0.633439302444458
Batch 20 loss: 0.256988525390625
Batch 21 loss: 0.26989659667015076
Batch 22 loss: 0.45568692684173584
Batch 23 loss: 0.3705233633518219
Batch 24 loss: 0.33448049426078796
Batch 25 loss: 0.4585626423358917
Batch 26 loss: 0.5318294763565063
Batch 27 loss: 0.3732

 13%|█▎        | 4/30 [12:10:16<86:37:00, 11993.11s/it] 

Batch 71 loss: 0.5144060254096985
Accumulated loss: 30.26707860827446

Epoch 5
------------------------
Batch 1 loss: 0.4141037166118622
Batch 2 loss: 0.5380562543869019
Batch 3 loss: 0.3907482922077179
Batch 4 loss: 0.3425721228122711
Batch 5 loss: 0.4089185893535614
Batch 6 loss: 0.4251788854598999
Batch 7 loss: 0.3137970268726349
Batch 8 loss: 0.40187448263168335
Batch 9 loss: 0.33579540252685547
Batch 10 loss: 0.4969669580459595
Batch 11 loss: 0.47049829363822937
Batch 12 loss: 0.38666635751724243
Batch 13 loss: 0.335449755191803
Batch 14 loss: 0.4300288259983063
Batch 15 loss: 0.3930479884147644
Batch 16 loss: 0.28847137093544006
Batch 17 loss: 0.3395904004573822
Batch 18 loss: 0.358744740486145
Batch 19 loss: 0.4070222079753876
Batch 20 loss: 0.4098818898200989
Batch 21 loss: 0.31509849429130554
Batch 22 loss: 0.4184629023075104
Batch 23 loss: 0.46934080123901367
Batch 24 loss: 0.3574407398700714
Batch 25 loss: 0.5192736983299255
Batch 26 loss: 0.5122268199920654
Batch 27 loss: 0

 17%|█▋        | 5/30 [34:31:48<254:49:13, 36694.14s/it]

Batch 71 loss: 0.6442673802375793
Accumulated loss: 26.86514151096344

Epoch 6
------------------------
Batch 1 loss: 0.2861570715904236
Batch 2 loss: 0.4272765517234802
Batch 3 loss: 0.30218395590782166
Batch 4 loss: 0.6334078311920166
Batch 5 loss: 0.6856504678726196
Batch 6 loss: 0.20052191615104675
Batch 7 loss: 0.4158209562301636
Batch 8 loss: 0.41207754611968994
Batch 9 loss: 0.4084608256816864
Batch 10 loss: 0.4034353494644165
Batch 11 loss: 0.5223625302314758
Batch 12 loss: 0.3256913721561432
Batch 13 loss: 0.4545782506465912
Batch 14 loss: 0.2193203568458557
Batch 15 loss: 0.35408562421798706
Batch 16 loss: 0.4419543147087097
Batch 17 loss: 0.35356876254081726
Batch 18 loss: 0.6239430904388428
Batch 19 loss: 0.3421770930290222
Batch 20 loss: 0.21873636543750763
Batch 21 loss: 0.14537855982780457
Batch 22 loss: 0.3503362238407135
Batch 23 loss: 0.3073289096355438
Batch 24 loss: 0.38601842522621155
Batch 25 loss: 0.32272976636886597
Batch 26 loss: 0.2556282579898834


In [17]:
with open('data/resnet18.pickle','wb') as fp:
    pickle.dump(model, fp)

## 6. Visualise model predictions <a name="6"></a>
Import necessary libraries for data-handling, plotting, modeling.
[↑ back to top ↑](#index)

## 7. Tune parameters <a name="7"></a>
Import necessary libraries for data-handling, plotting, modeling.
[↑ back to top ↑](#index)

## 8. Execute and evaluate <a name="8"></a>
Import necessary libraries for data-handling, plotting, modeling.
[↑ back to top ↑](#index)

## 9. Create predictor function<a name="9"></a>
Import necessary libraries for data-handling, plotting, modeling.
[↑ back to top ↑](#index)